In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb

In [ ]:
#load the IMDB dataset
max_features = 10000    #<-Vocab size
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)

In [3]:
#inspect our sample review
sample_review = x_train[0]

In [4]:
word_index = imdb.get_word_index()
reversed_word_index = {value: key for key,value in word_index.items()}
decoded_review = ' '.join([reversed_word_index.get(i-3,'?') for i in sample_review])
decoded_review


"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [5]:
#padding to our training and testing data
max_len = 500
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [6]:
#train simpleRNN
model = Sequential()
model.add(Embedding(max_features,128,input_length = max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [9]:
#create an instance for EarlyStopping callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor = 'val_loss',patience = 5,restore_best_weights=True)
earlystopping


In [10]:
#train the model
model.fit(x_train,
          y_train,
          epochs=10,batch_size = 32,
          validation_split = 0.2,
          callbacks=[earlystopping])

Epoch 1/10
625/625 [==============================] - 53s 84ms/step - loss: 57534.7656 - accuracy: 0.6605 - val_loss: 0.5634 - val_accuracy: 0.7056
Epoch 2/10
625/625 [==============================] - 51s 82ms/step - loss: 2.2185 - accuracy: 0.7878 - val_loss: 0.5906 - val_accuracy: 0.6698
Epoch 3/10
625/625 [==============================] - 51s 81ms/step - loss: 0.4799 - accuracy: 0.7882 - val_loss: 0.5091 - val_accuracy: 0.7586
Epoch 4/10
625/625 [==============================] - 51s 82ms/step - loss: 0.3623 - accuracy: 0.8614 - val_loss: 0.4867 - val_accuracy: 0.7822
Epoch 5/10
625/625 [==============================] - 51s 82ms/step - loss: 0.2761 - accuracy: 0.9020 - val_loss: 0.4851 - val_accuracy: 0.7952
Epoch 6/10
625/625 [==============================] - 51s 82ms/step - loss: 0.2191 - accuracy: 0.9252 - val_loss: 0.5148 - val_accuracy: 0.7948
Epoch 7/10
625/625 [==============================] - 52s 82ms/step - loss: 0.2392 - accuracy: 0.9068 - val_loss: 0.5419 - val_accur

In [11]:
model.save("simple_rnn_imdb.h5")

/Users/ravichoudhary/Desktop/genAI_projects/myenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
